In [25]:
import pandas as pd

pbp = pd.read_csv('../../Data/play-by-play/pbp_exp3.csv') #processing from exp 3 to exp4
particpation_19 = pd.read_csv('../../Data/nfl_verse_participation/pbp_participation_2019.csv')
particpation_20 = pd.read_csv('../../Data/nfl_verse_participation/pbp_participation_2020.csv')

In [26]:
particpation = pd.concat([particpation_19,particpation_20])
particpation.shape

(96547, 20)

In [27]:
# inner_join = pbp.merge(particpation[['old_game_id', 'play_id', 'offense_formation', 
# 'defenders_in_box']], on=['old_game_id', 'play_id'], how='inner')
# inner_join.shape #losing some rows, better to use left join

In [28]:
left_join = pbp.merge(particpation[['old_game_id', 'play_id', 'offense_formation', 
'defenders_in_box']], on=['old_game_id', 'play_id'], how='left')
left_join.shape

(88336, 46)

In [29]:
left_join.isna().sum().tail() #nulls from original participation

home_team_on_offense          0
home_team_on_defense          0
Player-Injured-On-Play        0
offense_formation         17769
defenders_in_box          16629
dtype: int64

In [30]:
# left_join[left_join['defenders_in_box'].isna()].to_csv('nulls_box_defenders_check.csv', index=False)
#looking at rows where defendrs in box is null
# offense_fornmation_nulls = left_join[left_join['offense_formation'].isna()]

In [31]:
left_join['offense_formation'].value_counts(dropna=False) #17000+ nulls

SHOTGUN       37814
NaN           17769
SINGLEBACK    17407
EMPTY          6263
I_FORM         5851
PISTOL         2256
JUMBO           752
WILDCAT         224
Name: offense_formation, dtype: int64

In [32]:
def fill_offense_formation(row):
    desc = row['Description'].upper()
    if 'PUNTS' in desc or 'PUNT' in desc:
        return 'PUNT'
    elif 'FIELD GOAL' in desc or 'EXTRA POINT' in desc:
        return 'FIELD_GOAL'
    elif 'SHOTGUN' in desc or 'PASS FORMATION' in desc:
        return 'SHOTGUN'
    elif 'KICKS' in desc or 'KICK FORMATION' in desc:
        return 'KICKOFF'
    elif 'KNEELS' in desc:
        return 'VICTORY FORMATION'
    else:
        return 'UNDER CENTER'

left_join['offense_formation'] = left_join['offense_formation'].fillna(left_join.apply(fill_offense_formation, axis=1)) #only applying function to rows where label is currently null

In [33]:
left_join['offense_formation'].value_counts(dropna=False) #new value counts (no nulls)

SHOTGUN              39037
SINGLEBACK           17407
EMPTY                 6263
I_FORM                5851
KICKOFF               5651
FIELD_GOAL            4789
PUNT                  4507
PISTOL                2256
VICTORY FORMATION      872
JUMBO                  752
UNDER CENTER           727
WILDCAT                224
Name: offense_formation, dtype: int64

In [34]:
left_join['defenders_in_box'] = left_join['defenders_in_box'].fillna(0) #secial teams plays, can label 0 players in box

In [35]:
left_join['Player-Injured-On-Play'] = left_join.pop('Player-Injured-On-Play') #move back to last column

In [36]:
left_join.to_csv('../../Data/play-by-play/pbp_exp4.csv', index=False)